# 1) Regression:
## Choose 4 different methods of regression 
### a) linear regression
### b) polinominal regression

In [ ]:
from random import uniform
import sys
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

print(sys.executable)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

In [ ]:
# Might be good idea to have a function for dataset analysis --> standardized output
# Just a prototype/idea for now
def analyse_dataset(frame, name='not-given'):
    print(f"Analysis of <{name}>")
    print('-'*40)
    print("Info:")
    print(frame.info())
    print('-'*40)
    print("Shape:")
    print(frame.shape)
    print('-'*40)
    print("Index:")
    print(frame.index)
    print('-'*40)
    print("Columns:")
    print(frame.columns)

In [ ]:
dataset_name = 'Video_Games_Sales_as_at_22_Dec_2016'
input_file = os.path.split(sys.path[0])[0] + '\\datasets\\'+dataset_name+'.csv'
print("Opening: ", input_file)
data = pd.read_csv(input_file,  sep = ',')#, header = 0)
print(data)

In [ ]:
# Quick analysis
print(f"Analysis of <{dataset_name}>")
print('-'*40)
print("Info:")
print(data.info())

print("Columns:")
print(data.columns)



In [ ]:
### PREPROCESSING ###

# Drop some columns for now
cols_to_drop = ['Publisher', 'Genre', 'Critic_Count', 'Platform', 'Year_of_Release', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales','User_Count', 'Developer', 'Rating', 'Name']
prep = data.drop(cols_to_drop, axis = 1)

# Fix missing values
# There are some 'tbd' values in the scores --> functionally equivalent to NaN so replace to NaN
prep = prep.replace('tbd', None)
# drop row containing < thresh number of observations = not NaN
# --> we only want rows with at least one valid entry in either User_Score or Critic_Score
prep = prep.dropna(subset= ['Critic_Score', 'User_Score'], thresh=1)
prep = prep.dropna(subset=['Global_Sales'], how='any')
print(prep)

# Scale User_Score to 0...100 (similiar to Critic_Score)
prep['User_Score'] = prep['User_Score'].astype('float').multiply(10)

# How to treat the remaining missing values in on of the Scores? 
# Replace with other if present +- a random value with sigma
sigma = 5
mod = uniform(-sigma,sigma)
prep.User_Score.where((~prep.User_Score.isnull()), round(prep.Critic_Score+mod, 2), inplace=True)
prep.Critic_Score.where((~prep.Critic_Score.isnull()), round(prep.User_Score+mod, 2), inplace=True)

# Remove outliers in sales
from scipy import stats
prep = prep[(np.abs(prep.Global_Sales) > 1)]
prep = prep[(np.abs(stats.zscore(prep)) < 3).all(axis=1)]

### OUTPUT FINAL DATAFRAME ###
print('#'*60)
print("Preprocessing output")
print('#'*60)
print(prep)
print(prep.info())

In [ ]:
plt.scatter(prep.User_Score, prep.Global_Sales)

In [ ]:
sns.lmplot("User_Score","Global_Sales",prep)